In [2]:
from ultralytics import YOLO
import cv2

x128 = YOLO("runs/segment/train6/weights/best.pt")
x256 = YOLO("runs/segment/train7/weights/best.pt")
x384 = YOLO("runs/segment/train8/weights/best.pt")
x512 = YOLO("runs/segment/train9/weights/best.pt")
x640 = YOLO("runs/segment/train10/weights/best.pt")
x768 = YOLO("runs/segment/train11/weights/best.pt")

models = [x128, x256, x384, x512, x640, x768]
models_names = ["128x128", "256x256", "384x384", "512x512", "640x640", "768x768"]

In [3]:
import os
import numpy as np

data_path = "C:/Users/Artem/bboxed_barcodes_dataset/Validation/images/"
output_path = "C:/Users/Artem/mipt2024s-5-Tiniakov-A-D/yolo_input_size_tests/"

images = os.listdir(data_path)

In [4]:
import shutil

for i in range(len(models)):
    for image in images:
        models[i].predict(data_path+image, save=True, save_txt=True)
    shutil.move("C:/Users/Artem/runs/segment/predict", output_path+models_names[i])


image 1/1 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\IMG_10050.jpg: 96x128 1 1d, 216.2ms
Speed: 3.0ms preprocess, 216.2ms inference, 90.0ms postprocess per image at shape (1, 3, 96, 128)
Results saved to runs\segment\predict
1 label saved to runs\segment\predict\labels

image 1/1 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\IMG_10090.jpg: 128x128 1 1d, 40.3ms
Speed: 2.9ms preprocess, 40.3ms inference, 3.0ms postprocess per image at shape (1, 3, 128, 128)
Results saved to runs\segment\predict
2 labels saved to runs\segment\predict\labels

image 1/1 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\IMG_1013 20.jpg: 128x128 1 2d, 13.9ms
Speed: 1.1ms preprocess, 13.9ms inference, 3.0ms postprocess per image at shape (1, 3, 128, 128)
Results saved to runs\segment\predict
3 labels saved to runs\segment\predict\labels

image 1/1 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\IMG_12580.jpg: 128x128 1 1d, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inferen

In [5]:
def calculate_iou(pred_mask, gt_mask):
    if len(pred_mask) == 0 or len(gt_mask) == 0:
        return 0
        
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    iou = intersection / union if union > 0 else 0
    return iou

In [6]:
from copy import deepcopy

mean_ious = []

for i in range(len(models_names)):
    ground_truth_masks = []
    predicted_masks = []
    
    path_to_predicted_labels = "C:/Users/Artem/mipt2024s-5-Tiniakov-A-D/yolo_input_size_tests/" + models_names[i] + "/predict/labels/"
    path_to_gt_labels = "C:/Users/Artem/bboxed_barcodes_dataset/Validation/labels/"
    path_to_images = "C:/Users/Artem/bboxed_barcodes_dataset/Validation/images/"
    
    images_names = os.listdir(path_to_images)
    
    for name in images_names:
        name = name[:-4]
        img = cv2.imread(path_to_images+name+".jpg")
        height, width, _ = img.shape
        pred_mask = np.zeros((height, width, 1), dtype=np.uint8)
        gt_mask = np.zeros((height, width, 1), dtype=np.uint8)
        predicted_points = []
        gt_points = []

        predicted_labels = os.listdir(path_to_predicted_labels)

        if name+".txt" in predicted_labels:
            with open(path_to_predicted_labels+name+".txt", 'r') as fin:
                data = fin.readline().split(' ')[1:]
                for i in range(int(len(data)/2)):
                    predicted_points.append([int(float(data[i*2])*width), int(float(data[i*2+1])*height)])

        with open(path_to_gt_labels+name+".txt", 'r') as fin:
            data = fin.readline().split(' ')[1:]
            for i in range(int(len(data)/2)):
                gt_points.append([int(float(data[i*2])*width), int(float(data[i*2+1])*height)])
    
        predicted_points = np.array(predicted_points)
        gt_points = np.array(gt_points)

        if len(predicted_points) > 0:
            predicted_masks.append(cv2.fillPoly(pred_mask, pts=[predicted_points], color=(255)))
        else:
            predicted_masks.append(pred_mask)

        if len(gt_points) > 0:
            ground_truth_masks.append(cv2.fillPoly(gt_mask, pts=[gt_points], color=(255)))
        else:
            predicted_masks.append(gt_mask)

    ious = [calculate_iou(predicted_masks[i], ground_truth_masks[i]) for i in range(len(predicted_masks))]
    mean_ious.append(np.mean(ious))

In [7]:
mean_ious

[0.6411110765976297,
 0.7950138789328041,
 0.7235186691661133,
 0.6853521360525955,
 0.7593636285489671,
 0.7021744387419286]